# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from pandas import DataFrame

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [2]:
def movie_title_url_actor():

##### 1. 기본 설정
    url = 'https://movie.naver.com/movie/running/current.nhn'
    # 네이버 상영영화 url (예매순으로 보여 줌!)
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html)
    
##### 2.상위 5개 영화의 제목& 주연배우 3인 추출
    
    tit = soup.select('dt[class="tit"] a', limit = 5) #limit로 5개만 추출합니다!
    act = soup.select('span[class="link_txt"]')
    
    titles = [] # 영화 제목 
    actors = [] # 영화별 주연배우 3인

    for i in range(5):
        list_in = []
        act_in = act[(3*i)+2]
        
        for j in act_in.select('a'):
            list_in.append(j.get_text())
            if len(list_in) == 3:
                break # 주연배우가 3명 이하인 경우의 오류 방지!
        titles.append(tit[i].get_text())
        actors.append(list_in)
        
    return(titles, actors)

### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [3]:
def get_grade():

##### 1. 기본 설정
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html)

##### 2. 평점 가져올 url 할당
    m = soup.select('dt[class="tit"] a', limit=5) # 상위 5개의 영화 받음
    
    m_url = []
    for i in range(len(m)):
        m_url.append("https://movie.naver.com" + m[i].attrs['href'])
    m_url # 5개 영화 각각의 페이지 주소를 불러 옴
    
##### 3. 평점 받아오기
    score1 = [] # 네티즌
    score2 = [] # 관람객
    score3 = [] # 기자/평론가

    for i in m_url:
        url = i
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html)
        
        review = soup.select('div[class="star_score"] em', limit=12)
        hb = []
    
        for j in range(len(review)):
            hb.append(review[j].get_text())
        score1.append(hb[0]+hb[1]+hb[2]+hb[3])
        score2.append(hb[4]+hb[5]+hb[6]+hb[7])
        score3.append(hb[8]+hb[9]+hb[10]+hb[11])
    
    return(score1, score2, score3)

### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

#### get_reviews 함수는
 1. 네이버 상영영화 페이지에서 정규표현식으로 각 영화별 6자리 코드를 추출한 후
 2. 각 영화별 평점 페이지 url 할당
 3. 영화별로 20개의 평점, 평점 작성자 닉네임, 리뷰 본문 받아오기
   
   순서로 작성했습니다 :)

In [4]:
def get_reviews():
    # 기본 설정
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html)

    
##### 1. 영화별 코드 받아오기 by 정규표현식 
    m_code = soup.select('dt[class="tit"] a', limit = 5)
     # m_code에는 아래와 같이 들어와 있으므로 
     # <a href="/movie/bi/mi/basic.nhn?code=181925">클로젯</a>
     # 각 영화별로 code를 추출합시당
        
    codes = []
    for i in range(len(m_code)):
        p = re.compile('[0-9]{6}')
        codes.append(p.findall(str(m_code[i])))

        
##### 2. 각 영화마다 평점 페이지 2개씩 추출 (한페이지에 리뷰가 10개니까!)
    review_url = []
    for i in range(len(codes)):
        for j in range(2):
            review_url.append("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + str(codes[i][0]) + "&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page=" + str(j+1))
            # 총 10(5*2)개의 페이지 주소 추출!


##### 3.1. 영화별 평점 추출
    indiv_score = [] # 영화당 20개씩, 총 100개의 평점 받을 list 생성
    
    for i in range(len(review_url)): 
        url = review_url[i]
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html)
    
        m_score = soup.select('div[class="star_score"]', limit = 10)
        for j in range(len(m_score)):
            indiv_score.append(m_score[j].get_text().strip())
    
    # 20개씩 묶어줍시다
    final_score = []
    for i in [0, 20, 40, 60, 80]:
        final_score.append(indiv_score[i:i+20])
        # 각 영화별 평점 20개 저장!


##### 3.2. 영화별 리뷰 & 닉네임 추출
####### 1) 리뷰& 닉네임 받아오기
    review_nick = []
    
    for i in range(len(review_url)): 
        url = review_url[i]
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html)
        
        re_ni = soup.select('div[class="score_reple"] span')
        for j in range(len(re_ni)):
            review_nick.append(re_ni[j].get_text().strip())

####### 2) 필요없는 인자 제거 
    # 함수 정의
    def remove_values(mylist, val):
        return [value for value in mylist if value != val]
    
    # 제거 
    review_nick = remove_values(review_nick, "관람객")# 리스트에서 "관람객"인 경우 제거!
    review_nick = remove_values(review_nick, "스포일러가 포함된 감상평입니다. 감상평 보기") # 필요없음!
    
####### 3) 길이가 200이 아닌 경우 중복 제거 
    from collections import OrderedDict
    
    review_nick = list(OrderedDict.fromkeys(review_nick))
    
    review = [] # 리뷰
    m_id = [] #개인평점
    for i in range(len(review_nick)):
        if i%2 == 0 :
            review.append(review_nick[i])
        else :
            m_id.append(review_nick[i])

    final_id = []
    final_review = []
    for i in [0, 20, 40, 60, 80]:
        final_id.append(m_id[i:i+20])
        final_review.append(review[i:i+20])
        # 각 영화별로 20개씩 저장!
    
    return(final_score, final_id, final_review)

### 4. 저장하기

In [5]:
def save():
    movie = DataFrame()
    movie['영화제목'] = data1[0]
    movie['주연배우 3인'] = data1[1]
    movie['평점'] = data2[0]    
    movie['개인 평점'] = data3[0]
    movie['닉네임/아이디'] = data3[1]
    movie['리뷰'] = data3[2]
    movie.to_pickle('영화 크롤링1')
    movie.to_excel('영화 크롤링2.xlsx')

### 5. 크롤링하기

In [6]:
data1 = movie_title_url_actor()

In [7]:
data2 = get_grade()

In [8]:
data3 = get_reviews()

In [9]:
save()